# 1. Calculating similarity
1) Euclidean score : 유클리드 거리에 반비례. 주로 유클리드 거리를 0~1로 정규화하고, '1-유클리드 거리'로 계산한다.<br>
2) Pearson score : 두 객체 간의 상관관계를 측정한 값으로, 공분산과 개별 표준편차로 산정하며, -1~+1 사이의 값을 갖는다. <br>

In [1]:
import json
import numpy as np

In [2]:
#euclidean_score fucntion
def euclidean_score(dataset, user1, user2) : 
    #check if user1 & user2 exist
    if user1 not in dataset : 
        raise TypeError("Cannot find "+user1+"in the dataset")
    if user2 not in dataset : 
        raise TypeError("Cannot find "+user2+"in the dataset")
        
    #common movie of user1 & user2
    common_movies = {}
    
    for item in dataset[user1] : 
        if item in dataset[user2] : 
            common_movies[item] = 1
    
    if not len(common_movies) : return 0 #공통 영화 없다면 점수는 0점으로 반환하고 함수 종료 
    
    squared_diff = []
    for item in dataset[user1] : 
        if item in dataset[user2] :
            squared_diff.append(np.square(dataset[user1][item] - dataset[user2][item])) #두 유저의 평점 차이 제곱
        euclidean_dist = np.sqrt(np.sum(squared_diff)) #유클리디언 거리
        return 1/(1+euclidean_dist)

In [3]:
#Pearson_score function
def pearson_score(dataset, user1, user2) : 
    #check if user1 & user2 exist
    if user1 not in dataset : 
        raise TypeError("Cannot find "+user1+"in the dataset")
    if user2 not in dataset : 
        raise TypeError("Cannot find "+user2+"in the dataset")
        
    #common movie of user1 & user2
    common_movies = {}
    
    for item in dataset[user1] : 
        if item in dataset[user2] : 
            common_movies[item] = 1
    
    if not len(common_movies) : return 0 #공통 영화 없다면 점수는 0점으로 반환하고 함수 종료 
    
    #각 유저별 공통 영화에 대한 평점 합
    user1_sum = np.sum([dataset[user1][item] for item in common_movies])
    user2_sum = np.sum([dataset[user2][item] for item in common_movies])
    #각 유저별 공통 영화에 대한 평점 제곱합
    user1_squared_sum = np.sum([np.square(dataset[user1][item]) for item in common_movies])
    user2_squared_sum = np.sum([np.square(dataset[user2][item]) for item in common_movies])
    
    #공통되는 영화의 두 유저 평점의 내적(=곱의 합)
    sum_of_products = np.sum([dataset[user1][item]*dataset[user2][item] for item in common_movies])
    
    #피어슨 상관도 점수를 위한 매개변수 계산
    Sxy = sum_of_products - (user1_sum*user2_sum/len(common_movies))
    Sxx = user1_squared_sum - np.square(user1_sum)/len(common_movies)
    Syy = user2_squared_sum - np.square(user2_sum)/len(common_movies)
    
    if Sxx*Syy==0 : return 0 #두 객체 중 하나라도 0이면(= 편차가 없다면) 점수를 구할 수 없으니 0을 반환하고 함수 종료.
    else : return Sxy/np.sqrt(Sxx*Syy)

#### 피어슨 상관계수 = 공분산/(분산1*분산2)
- 두 객체의 분산이 모두 0이 아닐 때 구할 수 있다. 
- 아래 코드를 보면, 얼핏 분산을 구하는 식과 비슷한데 조금 다르다.<br>
>Sxx = user1_squared_sum - np.square(user1_sum)/len(common_movies)<br>
>Syy = user2_squared_sum - np.square(user2_sum)/len(common_movies)<br>

- 근데 위 2개의 코드는 분산이라고 보기엔 조금 이상하다. 분산이라면 아래와 같은 식이었어야 한다.<br>
>Sxx = user1_squared_sum/len(common_movies) - np.square(user1_sum/len(common_movies))<br>
>Syy = user2_squared_sum/len(common_movies) - np.square(user2_sum/len(common_movies))<br>

- 공분산도 마찬가지로 미묘하게 다르다. (첫번째 식은 코드에서 주어진 식, 두번째 식이 일반적인 공분산을 구하는 식) <br>
>Sxy = sum_of_products - (user1_sum*user2_sum/len(common_movies)) <br>
>Sxy = sum_of_products/len(common_movies) - user1_sum*user2_sum/np.square(len(common_movies))<br>

    (피어슨 점수라 조금 다른건가....)

In [4]:
ratings_file = "ratings.json"
with open(ratings_file, 'r') as fp : 
    data = json.loads(fp.read())

In [5]:
data

{'Adam Cohen': {'Goodfellas': 4.5,
  'Roman Holiday': 3.0,
  'Scarface': 3.0,
  'The Apartment': 1.0,
  'Vertigo': 3.5},
 'Bill Duffy': {'Goodfellas': 4.5,
  'Scarface': 5.0,
  'The Apartment': 1.0,
  'Vertigo': 4.5},
 'Brenda Peterson': {'Goodfellas': 2.0,
  'Raging Bull': 1.0,
  'Roman Holiday': 4.5,
  'Scarface': 1.5,
  'The Apartment': 5.0,
  'Vertigo': 3.0},
 'Chris Duncan': {'Raging Bull': 4.5, 'The Apartment': 1.5},
 'Clarissa Jackson': {'Goodfellas': 2.5,
  'Raging Bull': 4.0,
  'Roman Holiday': 1.5,
  'Scarface': 4.5,
  'The Apartment': 1.0,
  'Vertigo': 5.0},
 'David Smith': {'Goodfellas': 4.5,
  'Raging Bull': 3.0,
  'Scarface': 4.5,
  'The Apartment': 1.0,
  'Vertigo': 4},
 'Julie Hammel': {'Goodfellas': 3.0, 'Roman Holiday': 4.5, 'Scarface': 2.5},
 'Samuel Miller': {'Goodfellas': 5.0,
  'Raging Bull': 5.0,
  'Roman Holiday': 1.0,
  'Scarface': 3.5,
  'The Apartment': 1.0}}

In [6]:
user1 = "David Smith"
user2 = "Bill Duffy"
score_types = ["Euclidean", "Pearson"]

In [7]:
for score_type in score_types : 
    if score_type == "Euclidean" : 
        print("\nEuclidean score")
        print(euclidean_score(data, user1, user2))
    else :
        print("\nPearson score:")
        print(pearson_score(data, user1, user2))


Euclidean score
0.666666666667

Pearson score:
0.99099243041


<br><hr><br>

# 2. Finding similar user with Collaborative-Filtering 
- 협업 필터링이라고 하는 것은 여러 객체(유저, 아이템, 혹은 컨텐츠)의 정보를 함께 활용해 거리가 먼 객체를 제거해나가는 것이기 때문이라고 한다. 다른 필터링 추천 방법(content-based filtering)과의 차이점은 유사도가 높은(거리가 가까운) 유저의 정보를 이용한다는 점이다.
- Collaborative filtering의 종류 : 
    1. content-based filtering : 현재 유저가 본 아이템과 유사한 아이템을 추천. 단점은 유저가 본 카테고리에 속하는 아이템만을 추천할 수 있다.
    2. user-based collaborative filtering : 현재 유저와 유사도가 높은 유저의 상품 목록을 추천한다. 본 절에서 예시로 사용하는 추천은 사실상 user-based collaborative filtering이라고 할 수 있다. 유저와의 유사도를 평가할 때 본 절의 예시처럼 제품에 대한 평가를 기준으로 평가할 수도 있고, 또 하나의 가장 흔한 방법은 성별/연령대/지역 등의 인구통계학적 정보를 이용할 수도 있다.
    3. item-based collaborative filtering : 현재 유저와 유사도가 높은 유저를 고르고, 그 유저의 상품 특성과 


- refer : 영화 추천을 위한 앙상블 모델

In [8]:
user = "Bill Duffy"

In [9]:
#피어슨 점수에 따라 상위 n명의 유저와 피어슨 점수 반환하는 함수
def find_similar_users(dataset, user, num_users) : 
    if user not in dataset : 
        raise TypeError("Cannot find "+user+" in the dataset")
        
    #user 본인이 아닌 유저에 한해, [유저, 피어슨거리]의 형태의 리스트를 원소로 갖는 리스트 생성
    scores = np.array([[x, pearson_score(dataset, user, x)] for x in dataset if x!=user]) #list-comprehension 짱짱

    #점수순으로 sorting한 뒤 그 인덱스를 저장하고 그 중 top n명만 뽑는다.
    scores_sorted = np.argsort(scores[:, 1][::-1]) #cf. flipud 쓰면 역순으로 정렬
    top_users = scores_sorted[:num_users] #top n user의 인덱스
    
    return scores[top_users]    

In [10]:
print("\nUsers similar to "+user+":\n")
similar_users = find_similar_users(data, user, 3)
print("User\t\t\tSimilarity score")
print("-"*40)
for item in similar_users : 
    print(item[0], "\t\t", round(float(item[1]),2))


Users similar to Bill Duffy:

User			Similarity score
----------------------------------------
David Smith 		 0.99
Chris Duncan 		 0.0
Julie Hammel 		 -1.0


<br><hr><br>

# 3. Recommendation system using similarity metric
- 사실 위 과정까지 끝났다면, 이건 별 거 없다... 툭 까놓고 앞서 구한 top n 유저가 본 영화 중에 본인이 안 본 영화만 리스트업하는 것

In [11]:
def get_recommendations(dataset, input_user) : 
    if input_user not in dataset : 
        raise TypeError("Cannot find "+input_user+" in the dataset")
    
    overall_scores = {}
    similarity_scores = {}
    
    for user in [x for x in dataset if x!=input_user] : 
        similarity_score = pearson_score(dataset, input_user, user)
        if similarity_score <= 0 : continue
        
        #input_user가 보지 않은 영화만 filtered_list에 저장한다.
        filtered_list = [x for x in dataset[user] if x not in dataset[input_user] or dataset[input_user][x]==0]
        
        for item in filtered_list : 
            if item in overall_scores : #overall_scores에 있으면(이미 한번 추천된 영화라면), 가중치*평점을 더해서 추천 점수를 갱신하고
                overall_scores[item] += dataset[user][item]*similarity_score
            else : #overall_scores에 없으면(아직 추천된 적 없는 영화라면), 새로운 key:value쌍을 사전에 추가한다. 
                overall_scores.update({item : dataset[user][item]*similarity_score})
                
            if item in similarity_scores : 
                similarity_scores[item] += similarity_score
            else : 
                similarity_scores.update({item : similarity_score})
                
    if not len(overall_scores) : 
        return ["No recommendations possible"]
    
    movie_scores = np.array([[score/similarity_scores[item], item] for item, score in overall_scores.items()])
    #위 리스트 컴프리헨션에서 item: 필터링된 영화리스트에 속한 영화 품목, score: 해당 영화품목에 대한 유저들의 가중점수의 합 을 의미한다.
    #결국 movie_scores = '[Σ(영화평점*피어슨점수)/Σ피어슨점수, 영화품목]'로 이루어진 numpy 배열이며, 내림차순으로 정렬해 영화를 추천한다.
    movie_scores = movie_scores[np.argsort(movie_scores[:,0])[::-1]]
    
    movie_recommendations = [movie for _, movie in movie_scores]
    return movie_recommendations

In [12]:
ratings_file = "ratings.json"
with open(ratings_file, 'r') as fp : 
    data = json.loads(fp.read())

In [13]:
user = "Chris Duncan"

In [14]:
print("\nMovie recommendations for "+user+":")
movies = get_recommendations(data, user)
for i, movie in enumerate(movies) : 
    print(str(i+1)+". "+movie)


Movie recommendations for Chris Duncan:
1. Vertigo
2. Scarface
3. Goodfellas
4. Roman Holiday


### 생각해볼 점...
- 평소 추천 시스템을 만들 일이 있을 때 새로운 유저가 들어올 때마다 유사도 테이블을 갱신해야 한다. 근데 실제로 CF 뿐만이 아니라 대부분의 추천 시스템이 그러한데, 혹시 아이디어 있으신 분?